# =============================================================

# RAG Implementation with MSSQL: Pilot

# =============================================================

### Libraries

In [1]:
# !pip install -U langchain langchain_experimental
# !pip install typing-extensions --upgrade

from langchain.llms import LlamaCpp
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain

### Database Connection in Langchain 

In [2]:
# Connection string
uri = "mssql+pyodbc://@DESKTOP-UM221JG\\SQLEXPRESS/Test?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
db = SQLDatabase.from_uri(uri, include_tables=["Employee"])

In [3]:
# Table information schema
print(db.table_info)


CREATE TABLE [Employee] (
	[EmployeeId] INTEGER NOT NULL, 
	[LastName] NVARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[FirstName] NVARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Title] NVARCHAR(30) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[ReportsTo] INTEGER NULL, 
	[BirthDate] DATETIME NULL, 
	[HireDate] DATETIME NULL, 
	[Address] NVARCHAR(70) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[City] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[State] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Country] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[PostalCode] NVARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Phone] NVARCHAR(24) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Fax] NVARCHAR(24) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Email] NVARCHAR(60) COLLATE SQL_Latin1_General_CP1_CI_AS NULL
)

/*
3 rows from Employee table:
EmployeeId	LastName	FirstName	Title	ReportsTo	BirthDate	HireDate	Address	Cit

### Loading Mistral-Instruct LLM 

In [4]:
# Defining the model specs for optimized performance 
llama_model_path = r"C:\Users\DELL\Desktop\KSIE\KSIE LLM Chatbot\Models\mistral-7b-instruct-v0.1.Q8_0.gguf"
model_n_ctx = 2048
model_n_batch = 512

# Initializing the LLM object
llm = LlamaCpp(model_path=llama_model_path, n_batch=model_n_batch, n_ctx=model_n_ctx, verbose=True)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


### Creating the Retreival Chain

In [5]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

### Execution 

In [6]:
# Example 1
db_chain.run("How many employees are there?")

C:\Users\DELL\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM [Employee]
SQLResult: [(8,)]
Answer:

Llama.generate: prefix-match hit


There are 8 employees in the company.
> Finished chain.


'There are 8 employees in the company.'

In [7]:
# Example 2
db_chain.run("In which city does Adams Andrew live?")



> Entering new SQLDatabaseChain chain...
In which city does Adams Andrew live?
SQLQuery:

Llama.generate: prefix-match hit


SELECT City 
FROM [Employee] 
WHERE [LastName] = 'Adams' AND [FirstName] = 'Andrew';
SQLResult: [('Edmonton',)]
Answer:

Llama.generate: prefix-match hit


Adams Andrew lives in Edmonton.
> Finished chain.


'Adams Andrew lives in Edmonton.'

In [8]:
# Example 3
db_chain.run("Which employee was hired most recently?")



> Entering new SQLDatabaseChain chain...
Which employee was hired most recently?
SQLQuery:

Llama.generate: prefix-match hit


SELECT TOP 1 * FROM [Employee] ORDER BY [HireDate] DESC
SQLResult: [(8, 'Callahan', 'Laura', 'IT Staff', 6, datetime.datetime(1968, 1, 9, 0, 0), datetime.datetime(2004, 3, 4, 0, 0), '923 7 ST NW', 'Lethbridge', 'AB', 'Canada', 'T1H 1Y8', '+1 (403) 467-3351', '+1 (403) 467-8772', 'laura@chinookcorp.com')]
Answer:

Llama.generate: prefix-match hit


Laura Callahan was hired most recently.
> Finished chain.


'Laura Callahan was hired most recently.'